# NLP Machine Translation

In this notebook, I've built a recurrent neural network that functions as part of an end-to-end machine translation pipeline. It accepts English text as input and returns the French translation.

- **Preprocess**: converted text to sequence of integers.
- **Models**: created models which accepts a sequence of integers as input and returns a probability distribution over possible translations
- **Prediction**: runs the model on English text and outputs French translation

In [6]:
import collections
import os
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14862894950804826142
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357236736
locality {
  bus_id: 1
}
incarnation: 16653488549066632062
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file

In [3]:
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [4]:
# Load English data
english_sentences = load_data('data/small_vocab_en')
# Load French data
french_sentences = load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [23]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))
    print()

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .



The sentences have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase. 

### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print(english_words_counter.most_common(10))
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print(french_words_counter.most_common(10))

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
[('is', 205858), (',', 140897), ('.', 129039), ('in', 75525), ('it', 75137), ('during', 74933), ('the', 67628), ('but', 63987), ('and', 59850), ('sometimes', 37746)]

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
[('est', 196809), ('.', 135619), (',', 123135), ('en', 105768), ('il', 84079), ('les', 65255), ('mais', 63987), ('et', 59851), ('la', 49861), ('parfois', 37746)]


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
We need to convert the text into sequences of integers using the following preprocessing methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

We'll turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. 

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    text_tokenized = tokenizer.texts_to_sequences(x)
    
    return text_tokenized, tokenizer

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length. We can use Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    return pad_sequences(x, maxlen=length, dtype='int32',
                  padding='post', truncating='post', value=0.)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)
print()
print('First English sentence:')
print(english_sentences[:1])
print(preproc_english_sentences[:1].flatten())
print()
print('First French sentence:')
print(french_sentences[:1])
print(preproc_french_sentences[:1].flatten())

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344

First English sentence:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']
[17 23  1  8 67  4 39  7  3  1 55  2 44  0  0]

First French sentence:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]
[ 35  34   1   8  67  37  11  24   6   3   1 112   2  50   0   0   0   0
   0   0   0]


### Split dataset

In [10]:
X = pad(preproc_english_sentences, max_french_sequence_length)
y = preproc_french_sentences

# Reshaping the input to work with when no Embedding layer
X_tmp = X.reshape((-1, preproc_french_sentences.shape[-2], 1))

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Models
In this section, I've experimented with various neural network architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, I've constructed a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation..

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: Simple RNN
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [13]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    model = Sequential()
    model.add(SimpleRNN(512, input_shape=input_shape[1:], return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax')) 
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Train the neural network
simple_rnn_model = simple_model(X_tmp.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
simple_rnn_model.fit(X_tmp, y, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
for i in range(3):
    print()
    print('Input: ', english_sentences[i])
    print('Label: ', french_sentences[i])
    print('Prediction: ', logits_to_text(simple_rnn_model.predict(np.reshape(X_tmp[i], (1,21,1)))[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s 89us/step - loss: 2.1078 - acc: 0.5329 - val_loss: nan - val_acc: 0.5922
Epoch 2/10
110288/110288 [==============================] - 9s 83us/step - loss: 1.4501 - acc: 0.6082 - val_loss: nan - val_acc: 0.6253
Epoch 3/10
110288/110288 [==============================] - 9s 83us/step - loss: 1.2939 - acc: 0.6345 - val_loss: nan - val_acc: 0.6472
Epoch 4/10
110288/110288 [==============================] - 9s 83us/step - loss: 1.2043 - acc: 0.6487 - val_loss: nan - val_acc: 0.6585
Epoch 5/10
110288/110288 [==============================] - 9s 83us/step - loss: 1.1438 - acc: 0.6600 - val_loss: nan - val_acc: 0.6663
Epoch 6/10
110288/110288 [==============================] - 9s 83us/step - loss: 1.0979 - acc: 0.6676 - val_loss: nan - val_acc: 0.6745
Epoch 7/10
110288/110288 [==============================] - 9s 83us/step - loss: 1.0633 - acc: 0.6716 - val_loss: nan - val_acc: 0.6

### Model 2: Simple RNN with Embedding
![RNN](images/embedding.png)
We've turned the words into ids, but there's a better representation of a word: word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In [14]:
from keras.layers import Embedding

In [15]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 128, input_shape=input_shape[1:]))
    model.add(SimpleRNN(512, return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax')) 
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Train the neural network
embed_rnn_model = embed_model(X.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
embed_rnn_model.fit(X, y, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
for i in range(3):
    print()
    print('Input: ', english_sentences[i])
    print('Label: ', french_sentences[i])
    print('Prediction: ', logits_to_text(embed_rnn_model.predict(np.reshape(X[i], (1,21)))[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s 94us/step - loss: 2.5552 - acc: 0.5122 - val_loss: nan - val_acc: 0.5811
Epoch 2/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.5205 - acc: 0.6350 - val_loss: nan - val_acc: 0.6914
Epoch 3/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.0218 - acc: 0.7383 - val_loss: nan - val_acc: 0.7804
Epoch 4/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.7257 - acc: 0.8071 - val_loss: nan - val_acc: 0.8337
Epoch 5/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.5384 - acc: 0.8466 - val_loss: nan - val_acc: 0.8616
Epoch 6/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.4389 - acc: 0.8692 - val_loss: nan - val_acc: 0.8791
Epoch 7/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.3810 - acc: 0.8827 - val_loss: nan - val_ac

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [16]:
from keras.layers import Bidirectional

In [17]:
def bidir_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(512, return_sequences=True), input_shape=input_shape[1:]))
    model.add(Dense(french_vocab_size, activation='softmax')) 
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Train the neural network
bidir_rnn_model = bidir_model(X_tmp.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
bidir_rnn_model.fit(X_tmp, y, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
for i in range(3):
    print()
    print('Input: ', english_sentences[i])
    print('Label: ', french_sentences[i])
    print('Prediction: ', logits_to_text(bidir_rnn_model.predict(np.reshape(X_tmp[i], (1,21,1)))[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 16s 142us/step - loss: 1.7719 - acc: 0.5845 - val_loss: nan - val_acc: 0.6298
Epoch 2/10
110288/110288 [==============================] - 15s 137us/step - loss: 1.2707 - acc: 0.6431 - val_loss: nan - val_acc: 0.6535
Epoch 3/10
110288/110288 [==============================] - 15s 137us/step - loss: 1.1534 - acc: 0.6657 - val_loss: nan - val_acc: 0.6711
Epoch 4/10
110288/110288 [==============================] - 15s 137us/step - loss: 1.0744 - acc: 0.6813 - val_loss: nan - val_acc: 0.6884
Epoch 5/10
110288/110288 [==============================] - 15s 137us/step - loss: 1.0125 - acc: 0.6928 - val_loss: nan - val_acc: 0.6972
Epoch 6/10
110288/110288 [==============================] - 15s 137us/step - loss: 0.9652 - acc: 0.7005 - val_loss: nan - val_acc: 0.7047
Epoch 7/10
110288/110288 [==============================] - 15s 137us/step - loss: 0.9276 - acc: 0.7067 - val_loss: nan -

### Model 4: Encoder-Decoder
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [18]:
from keras.layers import RepeatVector

In [20]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    model = Sequential()
    model.add(SimpleRNN(512, return_sequences=False, input_shape=input_shape[1:]))
    model.add(RepeatVector(output_sequence_length))
    model.add(SimpleRNN(512, return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax')) 
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Train the neural network
encdec_rnn_model = encdec_model(X_tmp.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
encdec_rnn_model.fit(X_tmp, y, batch_size=1024, epochs=10, validation_split=0.2)

for i in range(3):
    print()
    print('Input: ', english_sentences[i])
    print('Label: ', french_sentences[i])
    print('Prediction: ', logits_to_text(encdec_rnn_model.predict(np.reshape(X_tmp[i], (1,21,1)))[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 16s 148us/step - loss: 2.4932 - acc: 0.4758 - val_loss: nan - val_acc: 0.5559
Epoch 2/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.5977 - acc: 0.5811 - val_loss: nan - val_acc: 0.6012
Epoch 3/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.4130 - acc: 0.6096 - val_loss: nan - val_acc: 0.6189
Epoch 4/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.3319 - acc: 0.6246 - val_loss: nan - val_acc: 0.6323
Epoch 5/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.2807 - acc: 0.6331 - val_loss: nan - val_acc: 0.6350
Epoch 6/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.2533 - acc: 0.6371 - val_loss: nan - val_acc: 0.6380
Epoch 7/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.2385 - acc: 0.6387 - val_loss: nan -

### Model 5: Custom
This model incorporates both embedding and a bidirectional rnn

In [21]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 128, input_shape=input_shape[1:]))
    model.add(Bidirectional(SimpleRNN(512, return_sequences=True)))
    model.add(Dense(french_vocab_size, activation='softmax')) 
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Train the neural network
final_rnn_model = model_final(X.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
final_rnn_model.fit(X, y, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
for i in range(10):
    print()
    print('Input: ', english_sentences[i])
    print('Label: ', french_sentences[i])
    print('Prediction: ', logits_to_text(final_rnn_model.predict(np.reshape(X[i], (1,21)))[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 18s 159us/step - loss: 2.1634 - acc: 0.5310 - val_loss: nan - val_acc: 0.6495
Epoch 2/20
110288/110288 [==============================] - 17s 155us/step - loss: 1.1072 - acc: 0.7241 - val_loss: nan - val_acc: 0.7823
Epoch 3/20
110288/110288 [==============================] - 17s 156us/step - loss: 0.6782 - acc: 0.8198 - val_loss: nan - val_acc: 0.8489
Epoch 4/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.4455 - acc: 0.8743 - val_loss: nan - val_acc: 0.8950
Epoch 5/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.3211 - acc: 0.9056 - val_loss: nan - val_acc: 0.9152
Epoch 6/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.2549 - acc: 0.9231 - val_loss: nan - val_acc: 0.9306
Epoch 7/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.2116 - acc: 0.9356 - val_loss: nan -